In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from google.colab import drive
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Model

# Mount Google Drive
drive.mount('/content/drive')

print("TensorFlow Version:", tf.__version__)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
TensorFlow Version: 2.19.0


In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
CLASSES = ["acne", "pigmentation", "wrinkles"]
DATA_ROOT = "/content/drive/MyDrive/skincareapp/acne clean pigmentation wrinkles/"

# --- Load CSV and prepare file paths ---
df = pd.read_csv(os.path.join(DATA_ROOT, "labels.csv"))

# Create a 'filename' column with the full path to each image
df["filename"] = df["filename"].apply(lambda x: os.path.join(DATA_ROOT, x))

print("DataFrame Head:")
print(df.head())
print(f"\nTotal images found: {len(df)}")

DataFrame Head:
                                            filename  acne  wrinkles  \
0  /content/drive/MyDrive/skincareapp/acne clean ...     1         0   
1  /content/drive/MyDrive/skincareapp/acne clean ...     1         0   
2  /content/drive/MyDrive/skincareapp/acne clean ...     1         0   
3  /content/drive/MyDrive/skincareapp/acne clean ...     1         0   
4  /content/drive/MyDrive/skincareapp/acne clean ...     1         0   

   pigmentation  clean  
0             0      0  
1             0      0  
2             0      0  
3             0      0  
4             0      0  

Total images found: 5062


In [ ]:
train_val_df, test_df = train_test_split(
    df,
    test_size=0.15,
    random_state=42,
    stratify=df[CLASSES]
)

# Second split to get the final training and validation sets
train_df, val_df = train_test_split(
    train_val_df,
    test_size=0.15,  # 15% of the remaining 85%
    random_state=42,
    stratify=train_val_df[CLASSES]
)

print(f"Training samples:   {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Test samples:       {len(test_df)}")


#Calculate class weights for the custom loss function
#These counts are from the original notebook and are used in the weighted BCE loss
pos_counts = train_df[CLASSES].sum().values
neg_counts = len(train_df) - pos_counts

print("\nPositive class counts (acne, pigmentation, wrinkles):", pos_counts)

Training samples:   3656
Validation samples: 646
Test samples:       760

Positive class counts (acne, pigmentation, wrinkles): [1015  386  738]


In [ ]:
def parse_function(filename, labels):
    #Read the file
    image_string = tf.io.read_file(filename)
    #Decode the image
    image_decoded = tf.io.decode_jpeg(image_string, channels=3)
    #Convert to float32
    image = tf.image.convert_image_dtype(image_decoded, tf.float32)
    #Resize the image
    image_resized = tf.image.resize(image, IMG_SIZE)
    return image_resized, labels

def create_dataset(df, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices(
        (df["filename"].values, df[CLASSES].values.astype(np.float32))
    )
    dataset = dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

#Create the datasets
train_ds = create_dataset(train_df, BATCH_SIZE)
val_ds = create_dataset(val_df, BATCH_SIZE)
test_ds = create_dataset(test_df, BATCH_SIZE)

print("\n✅ tf.data pipelines created successfully.")


✅ tf.data pipelines created successfully.


In [ ]:
#Hyperparameters for the EANet Model
PATCH_SIZE = 16
EMBEDDING_DIM = 256
MLP_DIM = 512
DIM_COEFFICIENT = 4
NUM_HEADS = 8
ATTENTION_DROPOUT = 0.2
PROJECTION_DROPOUT = 0.2
NUM_TRANSFORMER_BLOCKS = 4

print(f"Image Size: {IMG_SIZE[0]} X {IMG_SIZE[1]} = {IMG_SIZE[0] * IMG_SIZE[1]}")
print(f"Patch size: {PATCH_SIZE} X {PATCH_SIZE} = {PATCH_SIZE**2} ")
NUM_PATCHES = (IMG_SIZE[0] // PATCH_SIZE) ** 2
print(f"Patches per image: {NUM_PATCHES}")


# Custom Layer to Extract Patches
class PatchExtractor(layers.Layer):
    def __init__(self, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.patch_size = patch_size

    def call(self, images):
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        return patches

    # This method tells Keras how to save the layer's configuration
    def get_config(self):
        config = super().get_config()
        config.update({"patch_size": self.patch_size})
        return config


#EANet Helper Modules
class ExternalAttention(layers.Layer):
    def __init__(self, dim, num_heads, dim_coefficient=4, attention_dropout=0.2, projection_dropout=0.2):
        super().__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.dim_coefficient = dim_coefficient
        self.linear_q = layers.Dense(dim * dim_coefficient)
        self.linear_k = layers.Dense(dim * dim_coefficient)
        self.linear_v = layers.Dense(dim * dim_coefficient)
        self.linear_out = layers.Dense(dim)
        self.softmax = layers.Softmax(axis=-1)
        self.attention_drop = layers.Dropout(attention_dropout)
        self.projection_drop = layers.Dropout(projection_dropout)

    def call(self, inputs):
        q = self.linear_q(inputs)
        k = self.linear_k(inputs)
        v = self.linear_v(inputs)

        q = tf.reshape(q, (-1, q.shape[1], self.num_heads, self.dim_coefficient))
        k = tf.reshape(k, (-1, k.shape[1], self.num_heads, self.dim_coefficient))
        v = tf.reshape(v, (-1, v.shape[1], self.num_heads, self.dim_coefficient))

        q = tf.transpose(q, perm=[0, 2, 1, 3])
        k = tf.transpose(k, perm=[0, 2, 1, 3])
        v = tf.transpose(v, perm=[0, 2, 1, 3])

        attention = self.softmax(tf.matmul(q, k, transpose_b=True) / tf.math.sqrt(float(self.dim_coefficient)))
        attention = self.attention_drop(attention)

        out = tf.matmul(attention, v)
        out = tf.transpose(out, perm=[0, 2, 1, 3])
        out = tf.reshape(out, (-1, out.shape[1], self.dim * self.dim_coefficient))

        out = self.linear_out(out)
        out = self.projection_drop(out)
        return out

class TransformerBlock(layers.Layer):
    def __init__(self, embedding_dim, num_heads, mlp_dim, drop_rate=0.2, **kwargs):
        super().__init__(**kwargs)
        self.attention = ExternalAttention(dim=embedding_dim, num_heads=num_heads)
        self.norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.norm2 = layers.LayerNormalization(epsilon=1e-6)

        # Defining the MLP layers once inside __init__ using a Sequential model
        self.mlp = tf.keras.Sequential([
            layers.Dense(mlp_dim, activation=tf.nn.gelu),
            layers.Dropout(drop_rate),
            layers.Dense(embedding_dim),
            layers.Dropout(drop_rate)
        ])

    def call(self, x):
        attn_output = self.attention(self.norm1(x))
        x = layers.add([x, attn_output])
        mlp_output = self.mlp(self.norm2(x))
        return layers.add([x, mlp_output])

#Function to Build the Full EANet Model (No changes needed here)
def build_eanet(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    patches = PatchExtractor(PATCH_SIZE)(inputs)
    patch_dims = patches.shape[-1]
    patches = layers.Reshape((NUM_PATCHES, patch_dims))(patches)
    patch_embedding = layers.Dense(units=EMBEDDING_DIM)(patches)
    positions = tf.range(start=0, limit=NUM_PATCHES, delta=1)
    position_embedding = layers.Embedding(
        input_dim=NUM_PATCHES, output_dim=EMBEDDING_DIM
    )(positions)
    x = patch_embedding + position_embedding

    for _ in range(NUM_TRANSFORMER_BLOCKS):
        x = TransformerBlock(
            embedding_dim=EMBEDDING_DIM,
            num_heads=NUM_HEADS,
            mlp_dim=MLP_DIM
        )(x)

    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(num_classes, activation="sigmoid")(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

Image Size: 224 X 224 = 50176
Patch size: 16 X 16 = 256 
Patches per image: 196


In [ ]:
#Build the model
tf.keras.backend.clear_session() # Clear previous models from memory
eanet_model = build_eanet(
    input_shape=IMG_SIZE + (3,),
    num_classes=len(CLASSES)
)
eanet_model.summary()


#Custom Weighted Binary Cross-Entropy Loss (from your notebook)
def weighted_bce(y_true, y_pred, smooth=0.05):
    y_true = y_true * (1.0 - smooth) + 0.5 * smooth
    bce = tf.keras.backend.binary_crossentropy(y_true, y_pred)

    # Using the calculated pos_counts from Cell 3
    pos = tf.constant(pos_counts, dtype=tf.float32)
    neg = len(train_df) - pos

    w_pos = neg / tf.maximum(pos, 1.0)
    w_neg = tf.ones_like(pos)

    weights = y_true * w_pos + (1.0 - y_true) * w_neg
    return tf.reduce_mean(bce * weights)

#Compile the EANet model
eanet_model.compile(
    optimizer=tf.keras.optimizers.AdamW(1e-4, weight_decay=1e-5),
    loss=weighted_bce,
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name="acc", threshold=0.5),
        tf.keras.metrics.AUC(name="auc", multi_label=True),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall")
    ]
)

#Callbacks
EANET_MODEL_PATH = os.path.join(DATA_ROOT, "eanet_skin_model.keras")
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_auc", mode="max", patience=7, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_auc", mode="max", factor=0.5, patience=3, min_lr=1e-6),
    tf.keras.callbacks.ModelCheckpoint(EANET_MODEL_PATH, monitor="val_auc", mode="max", save_best_only=True)
]

#Train the model
print("\nStarting EANet model training...")
EPOCHS = 50
history_eanet = eanet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

print(f"\n✅ Training complete. Best EANet model saved to {EANET_MODEL_PATH}")

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ patch_extractor                 │ (None, 14, 14, 768)    │             0 │
│ (PatchExtractor)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 196, 768)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 196, 256)       │       196,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ add (Add)                       │ (None, 196, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ (None, 196, 256)       │     1,315,840 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_1             │ (None, 196, 256)       │     1,315,840 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_2             │ (None, 196, 256)       │     1,315,840 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_3             │ (None, 196, 256)       │     1,315,840 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,460,995 (20.83 MB)

 Trainable params: 5,460,995 (20.83 MB)

 Non-trainable params: 0 (0.00 B)


Starting EANet model training...
Epoch 1/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 126s 812ms/step - acc: 0.7010 - auc: 0.7437 - loss: 1.0479 - precision: 0.3580 - recall: 0.6599 - val_acc: 0.7214 - val_auc: 0.8867 - val_loss: 0.9696 - val_precision: 0.3990 - val_recall: 0.8466 - learning_rate: 1.0000e-04
Epoch 2/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 73s 632ms/step - acc: 0.8074 - auc: 0.8774 - loss: 0.7838 - precision: 0.5023 - recall: 0.7941 - val_acc: 0.7183 - val_auc: 0.9022 - val_loss: 1.0183 - val_precision: 0.3953 - val_recall: 0.8386 - learning_rate: 1.0000e-04
Epoch 3/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 73s 634ms/step - acc: 0.8299 - auc: 0.9028 - loss: 0.7159 - precision: 0.5406 - recall: 0.8260 - val_acc: 0.7472 - val_auc: 0.9200 - val_loss: 0.9744 - val_precision: 0.4257 - val_recall: 0.8492 - learning_rate: 1.0000e-04
Epoch 4/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 74s 640ms/step - acc: 0.8545 - auc: 0.9218 - loss: 0.6643 - precision: 0.5859 - recall: 0.8540 - val_acc: 0.7595 - val_auc: 0.9280 - val

In [ ]:
custom_objects = {
    "PatchExtractor": PatchExtractor,
    "TransformerBlock": TransformerBlock,
    "ExternalAttention": ExternalAttention,
    "weighted_bce": weighted_bce
}

#Loading the Model
MODEL_PATH = "/content/drive/MyDrive/skincareapp/acne clean pigmentation wrinkles/eanet_skin_model.keras"
print(f"Loading model from: {MODEL_PATH}")

loaded_model = tf.keras.models.load_model(MODEL_PATH, custom_objects=custom_objects)
print("✅ Model loaded successfully!")


#Evaluation on the Test Set
print("\nEvaluating the final model on the unseen test set...")
test_results = loaded_model.evaluate(test_ds)

print("\nFinal Test Set Evaluation Results")
for metric, value in zip(loaded_model.metrics_names, test_results):
    print(f"{metric}: {value:.4f}")

Loading model from: /content/drive/MyDrive/skincareapp/acne clean pigmentation wrinkles/eanet_skin_model.keras


/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'transformer_block', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'transformer_block_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'transformer_block_2', howev

✅ Model loaded successfully!

Evaluating the final model on the unseen test set...
24/24 ━━━━━━━━━━━━━━━━━━━━ 275s 11s/step - acc: 0.9492 - auc: 0.9793 - loss: 0.5812 - precision: 0.8852 - recall: 0.8398

Final Test Set Evaluation Results
loss: 0.6127
compile_metrics: 0.9461
